In [ ]:
from data_processing.dataset import load_dataset, get_impulses
from data_processing.obj2py import read_get

In [ ]:
import PIL
import numpy as np
import matplotlib.pyplot as plt
import vtk
from vtk.util.numpy_support import vtk_to_numpy
from tqdm import tqdm
from scipy.optimize import minimize
from vedo import *
import meshio
import seaborn as sns
import pyvista as pv
from data_processing.mesh_to_array import *
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
from hydra import compose, initialize
from omegaconf import OmegaConf

# Define the discrete values and corresponding grayscale values
values = [0, 1, 2, 3, 4, 5]
grayscale_values = np.linspace(0, 1, len(values))  # Grayscale values range from 0 to 1

# Create a colormap dictionary
colormap_dict = {}
for value, grayscale_value in zip(values, grayscale_values):
    colormap_dict[value] = grayscale_value

# Create a LinearSegmentedColormap object
cmap = LinearSegmentedColormap.from_list('custom_colormap', [(val, val, val) for val in grayscale_values], N=len(values))


# Show slice from .nas

In [ ]:
from data_processing.preprocessing import load_mask_target_electrodes
import matplotlib.pyplot as plt

In [ ]:
mesh_data = load_mask_target_electrodes('/home/nibdombe/deep_eit/data/raw/case_TCIA_211_0', resolution=512)

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(30, 10))
for i in range(4):
    axes[0,i].imshow(mesh_data[0][i])
    axes[0,i].axis('off')
    axes[0,i].scatter(mesh_data[-1][i][:,0], mesh_data[-1][i][:,1], marker='x', c='r')
    axes[1,i].imshow(mesh_data[1][i])
    axes[1,i].axis('off')
    axes[1,i].scatter(mesh_data[-1][i][:,0], mesh_data[-1][i][:,1], marker='x', c='r')



# Show slices from .npz

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = np.load('/home/nibdombe/deep_eit/data/processed/case_TCIA_104_0.npz')

In [ ]:
targets = data['targets']
electrodes = data['electrodes']

In [ ]:

fig, axes = plt.subplots(2, 4, figsize=(30, 10))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(targets[i])
    ax.axis('off')
    ax.scatter(electrodes[i][:,0], electrodes[i][:,1], marker='x', c='r', cmap='coolwarm')
    ax.scatter(get_impulses(electrodes[i])[:,0], get_impulses(electrodes[i])[:,1], marker='x', c='r', cmap='coolwarm')

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# Define the colors and their positions in the colormap
colors = [(1., 1., 1),   # blue
          (0.1, 0.1, 1), # light blue
          (1, 0.8, 0.4), # yellow
          (0.8, 0, 0)]     # red
positions = [0.0, 0.05, 0.2, 1.0]

# Create the colormap
cmap = LinearSegmentedColormap.from_list('custom_colormap', list(zip(positions, colors)))



In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(30, 10))
for i, ax in enumerate(axes.flatten()):
    sns.heatmap(targets[i].squeeze(), cmap=cmap, ax=ax)
    ax.axis('off')
    ax.scatter(electrodes[i][:,0], electrodes[i][:,1], marker='x', c='r', legend='Electrodes')

# Load from datasets

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dataset = torch.load('/home/nibdombe/deep_eit/data/datasets/train_dataset.pt')

In [ ]:
electrodes = dataset.electrodes

In [ ]:
e = electrodes[0,:,:2]
p = dataset.points
imp = dataset.impulses[0][:,:2]
t = dataset.targets[0].reshape(-1,1)
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(p[:,0], p[:,1], marker='x', c=t, s=.1)
ax.scatter(e[:,0], e[:,1], marker='x', c='r')
ax.scatter(imp[:,0], imp[:,1], marker='x', c='b')

In [ ]:
e_rot = torch.matmul(e, dataset._random_rotation_matrix(rotation_angle=135))
p_rot = torch.matmul(p, dataset._random_rotation_matrix(rotation_angle=135))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(p_rot[:,0], p_rot[:,1], marker='x', c=t, s=0.1)
ax.scatter(e_rot[:,0], e_rot[:,1], marker='x', c='r')


# Load model and process test data

In [ ]:
with initialize(version_base=None, config_path="cfg", job_name="test"):
    cfg = compose(config_name="config")


# Compare generated targets with Anita's targets

In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
toy = np.load('/work/deep_eit/data/processed/case_TCIA_53_0.npz', allow_pickle=True)

In [ ]:
toy['targets'].shape

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.heatmap(toy['targets'][0].squeeze(), ax=axes[0])
axes[0].axis('off')
axes[1].imshow(toy['masks'][0])

In [ ]:
anita_target = sio.loadmat('/work/deep_eit/data/raw/case_TCIA_53_0/targets/level_1.mat')['slice_z1']
anita_target = np.where(np.isnan(anita_target), 0, anita_target)
anita_target = np.moveaxis(anita_target, 0, 1)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.heatmap(toy['targets'][0].squeeze(), ax=axes[0], vmax=1)
axes[0].axis('off')
sns.heatmap(anita_target, ax=axes[1], vmax=1)
axes[1].axis('off')


In [ ]:
anita_target.shape

In [ ]:
np.unique(anita_target)

In [ ]:
import os


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision.transforms.functional as TF

In [ ]:
case_flip = np.load('/home/nibdombe/deep_eit/data/processed/case_TCIA_298_0.npz')
case_correct = np.load('/home/nibdombe/deep_eit/data/processed/case_TCIA_2_0.npz')

In [ ]:
img_flip = case_flip['targets'][4]
img_correct = case_correct['targets'][5]

In [ ]:
img_flip_flip = np.flip(img_flip, axis=0)
img_flip_flip_flip = np.flip(img_flip_flip, axis=1)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 5))
axes[0].imshow(img_flip.squeeze())
axes[1].imshow(img_flip_flip.squeeze())
axes[2].imshow(img_flip_flip_flip.squeeze())
axes[3].imshow(img_correct.squeeze())